# odinson
Retrieve original Marvel Comics editions for stories published in old brazilian comics

##### 2021.11.14

---

## Souces

[Guia dos Quadrinhos](http://www.guiadosquadrinhos.com/)
- [Heróis da TV](http://www.guiadosquadrinhos.com/edicao/herois-da-tv-2-serie-n-1/htv0302/6274)
- [Superaventuras Marvel](http://www.guiadosquadrinhos.com/edicao/superaventuras-marvel-n-1/sam0301/7927)
- [Almanaque Premiere Marvel](http://www.guiadosquadrinhos.com/edicao/almanaque-premiere-marvel-n-1/apm02001/18611)

---


## 1. Develop retrieving:
- edition
    - id="nome_titulo_lb"
- ~cover image? link?~
    - id="ampliar_capa"
- story (portuguese)
    - all: id="texto_pag_detalhe"
- main character
- original publication
- year
- original title

In [141]:
urlroot = "http://www.guiadosquadrinhos.com/"
# urledition = "edicao/herois-da-tv-2-serie-n-1/htv0302/6274"
urledition = "edicao/superaventuras-marvel-n-1/sam0301/7927"

In [153]:
urlroot + urledition

'http://www.guiadosquadrinhos.com/edicao/superaventuras-marvel-n-1/sam0301/7927'

### 1.1 dev scrape main #1 page

In [143]:
# dev scraping
import requests
from bs4 import BeautifulSoup

URL = urlroot + urledition
page = requests.get(URL)

thesoup = BeautifulSoup(page.content, "html.parser")

#### 1.1.1 Edition

In [90]:
# Edition
def edition(asoup):
    ed = asoup.find(id="nome_titulo_lb")
    edl = ed.contents
    edl = [f"{i}".strip() for i in edl]
    edl = [i.split(">")[1] if ">" in i else i for i in edl]
    edl = [i.split("<")[0] if "<" in i else i for i in edl]
    edl = [i.replace("n°", "").strip() for i in edl if i != '']
    return edl

In [126]:
ed = edition(thesoup)
ed

['Heróis da TV', '2ª Série -', '1']

#### 1.1.2 Edition Contents

In [92]:
def contents(asoup):
    results = asoup.find(id="texto_pag_detalhe")
    results = results.text
    return results

In [93]:
editioncontents = contents(thesoup)
print(editioncontents)


Crédito da capa e editor

Editor:  Dorival Vitor Lopes - ‘Dvl’,  Helcio de Carvalho

Primeira aparição de personagens nesta edição: 
Mercenário II
A lei do terror
Arco: Marcado Para Morrer  - Parte 2
Personagens:  Demolidor,  Mercenário II,  Viúva Negra II,  Ben Urich,  Heather Glenn,  Tucão,  Lark Logan,  Foggy Nelson,  Becky Blake
Roteiro:  Roger McKenzie Desenho:  Frank Miller Arte-Final:  Klaus Janson Cores:  Cleusa M. C. Acosta Letrista:  Edison Gasparim Tradutor:  Jotapê Martins Editor original:  Allen "Al" Milgrom,  Jim Shooter,  Mary Jo Duffy
Publicada originalmente em   Daredevil (1964) n° 160/1979 - Marvel Comics
             
Título original: "In the hands of Bullseye".Maxwell Glenn está morto. O namoro de Matt Murdock e Heather Glenn fica abalado quando ele se recusa a deixar de ser o Demolidor para ficar com ela. Deprimido e precisando desabafar, o herói decide fazer uma visita à sua velha parceira, e outrora amante, Viúva Negra, recém-chegada à Nova York. Mas, ao chegar 

#### 1.1.3 Stories

In [94]:
# All stories (individual)
def stories(asoup):
    # page section
    results = asoup.find(id="texto_pag_detalhe")
    # individual stories
    stories = results.find_all("div", class_="historia")
    stories = [i.text for i in stories]
    return stories

In [95]:
stories = stories(thesoup)
stories

['A lei do terror',
 'Na beira do abismo',
 'O habitante das trevas',
 'O herói que veio do inferno']

#### 1.1.4 Story Details

In [96]:
def storydetails(contents, stories, storynumber):
    # return details only if story exists
    if (storynumber == 0 or storynumber > len(stories)):
        storydetails = None
    # for any story before the last
    elif storynumber < len(stories):
        storydetails = contents.split(stories[storynumber-1])[1].split(stories[storynumber])[0]
    # for the last story
    elif storynumber == len(stories):
        storydetails = contents.split(stories[storynumber-1])[1]
    return storydetails

In [118]:
thestorydetail = storydetails(editioncontents, stories, 4)
print(thestorydetail)


Personagens:  Luke Cage,  Noah Burstein,  Shades,  Quirt,  Albert "Billy Bob" Rackham,  Comanche
Roteiro:  Archie Goodwin Desenho:  John Romita,  George Tuska Arte-Final:  John Romita,  Billy Graham Cores:  Rita de Carvalho Letrista:  José Luiz Torres Pinto Tradutor:  Jotapê Martins Editor original:  Stan Lee
Publicada originalmente em   Hero For Hire (1972) n° 1/1972 - Marvel Comics
             
Título original "Out of hell -- a hero!".A origem de Cage: incriminado e condenado à prisão, ele concorda em ser cobaia de um experimento na penitenciária. Mas o teste é sabotado por um guarda corrupto e nosso herói quase morre. Depois, ele descobre que o incidente lhe conferiu uma pele à prova de balas. Saindo da cadeia, ele, valendo-se do novo poder, resolve tirar vantagem, estabelecendo um escritório e passando a ser herói de aluguel.  21 páginas (das 23 originais).21 Páginas


               
               

              


#### 1.1.5 Main Character

In [100]:
def maincharacter(astorydetail):
    character = astorydetail.split(",")[0].split("Personagens:")[1].strip()
    return character

In [119]:
thecharacter = maincharacter(thestorydetail)
thecharacter

'Luke Cage'

#### 1.1.6 Original Title

In [102]:
def originaltitle(astorydetail):
    title = astorydetail.split("Publicada originalmente em")[1].split("(")[0].strip()
    return title

In [120]:
theoriginal = originaltitle(thestorydetail)
theoriginal

'Hero For Hire'

#### 1.1.7 Original Number

In [104]:
def originalnumber(astorydetail):
    issue = astorydetail.split("Publicada originalmente em")[1].split(")")[1].replace("n°\xa0", "").split("/")[0].strip()
    return issue

In [121]:
thenumber = originalnumber(thestorydetail)
thenumber

'1'

#### 1.1.8 Original Year

In [106]:
def originalyear(astorydetail):
    year = astorydetail.split("Publicada originalmente em")[1].split(")")[1].replace("n°\xa0", "").split("/")[1].split()[0].strip()
    return year

In [123]:
theyear = originalyear(thestorydetail)
theyear

'1972'

## 2 retrieve list of links for all editions

In [150]:
def navigator(asoup):
    outlist = []
    nav = asoup.find(id="d0")
    for link in nav.find_all("a"):
        outlist.append(f'{link["href"].replace("../", "")}')
    return outlist

In [151]:
alleditions = navigator(thesoup)

In [152]:
alleditions

['edicao/superaventuras-marvel-n-2/sam0301/7928',
 'edicao/superaventuras-marvel-n-3/sam0301/7929',
 'edicao/superaventuras-marvel-n-4/sam0301/7930',
 'edicao/superaventuras-marvel-n-5/sam0301/7931',
 'edicao/superaventuras-marvel-n-6/sam0301/7932',
 'edicao/superaventuras-marvel-n-7/sam0301/7933',
 'edicao/superaventuras-marvel-n-8/sam0301/7934',
 'edicao/superaventuras-marvel-n-9/sam0301/7935',
 'edicao/superaventuras-marvel-n-10/sam0301/7936',
 'edicao/superaventuras-marvel-n-11/sam0301/7937',
 'edicao/superaventuras-marvel-n-12/sam0301/7938',
 'edicao/superaventuras-marvel-n-13/sam0301/7939',
 'edicao/superaventuras-marvel-n-14/sam0301/7940',
 'edicao/superaventuras-marvel-n-15/sam0301/7941',
 'edicao/superaventuras-marvel-n-16/sam0301/7942',
 'edicao/superaventuras-marvel-n-17/sam0301/7943',
 'edicao/superaventuras-marvel-n-18/sam0301/7944',
 'edicao/superaventuras-marvel-n-19/sam0301/7945',
 'edicao/superaventuras-marvel-n-20/sam0301/7946',
 'edicao/superaventuras-marvel-n-21/sam

## 3. Save CSV and print log

In [1]:
dict1 = {1:("a", "b"), 2:("c", "d")}

In [2]:
dict1[2][0]

'c'